In [ ]:
import findspark
import plotly.graph_objects as go
from pyspark import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql import SQLContext


findspark.init()
sc = SparkContext("local", "first app")
spark = SparkSession(sc)
sqlContext = SQLContext(sc)

In [ ]:
archivo_entrada = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true', quote='"', delimiter=',').load("entradas/Covid19.csv")
rddfilter = archivo_entrada.rdd.map(tuple)

In [ ]:
rddCovid= rddfilter.map(lambda word: (word[7],word[5],word[6],word[2]))
rddCovid.take(10)

In [ ]:
rddOrder = rddCovid.filter(lambda x: "Guatemala" in x[0] and x[3]==8)
print("Resultados filtrados-> %s" % rddOrder.collect())

In [ ]:
sumCases = rddOrder.map(lambda x: x[1]).sum()
sumDeaths = rddOrder.map(lambda x: x[2]).sum()

rddNames = sc.parallelize(["Casos","Muertes"])
rddTotals = sc.parallelize([sumCases, sumDeaths])
print("Nombres -> %s" % rddNames.collect())
print("Valores -> %s" % rddTotals.collect())

In [ ]:
fig = go.Figure(data=go.Pie(labels=rddNames.collect(),values=rddTotals.collect()))
fig.update_layout(title_text='Casos y muertes Covid Guatemala en Agosto',title_font_size=30)
fig.update_traces(hoverinfo='label+percent', textinfo='value', textfont_size=20)
fig.write_html('R3_A3.html', auto_open=True)